<a href="https://colab.research.google.com/github/gabrielgranados97/Procesamiento-de-lenguaje-natural-2022/blob/main/Primera_tarea_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Primera tarea

**Lectura del set de datos**

In [ ]:
import numpy as np
import os
import gzip

def load_set(ruta, tipo='train'):

  ruta_categorias = os.path.join(ruta, '%s-labels-idx1-ubyte.gz' % tipo)
  ruta_imagenes = os.path.join(ruta, '%s-images-idx3-ubyte.gz' % tipo)
    
  with gzip.open(ruta_categorias, 'rb') as rut_cat:
      etiquetas = np.frombuffer(rut_cat.read(), dtype=np.uint8, offset=8)

  with gzip.open(ruta_imagenes, 'rb') as rut_imgs:
      imagenes = np.frombuffer(rut_imgs.read(), dtype=np.uint8, offset=16).reshape(len(etiquetas), 784)

  return imagenes, etiquetas

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

ruta = '/content/gdrive/MyDrive/Colab Notebooks/fashion_mnist_data'

X_train, Y_train = load_set(ruta, tipo='train')
X_test, Y_test = load_set(ruta, tipo='test')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Reajustar el tamaño de los datos**

In [ ]:
X_train = X_train[0:59904:]
X_test = X_test[0:9984:]
Y_train = Y_train[0:59904:]
Y_test = Y_test[0:9984]

X_train = np.reshape(X_train,(59904,28,28,1))
X_test = np.reshape(X_test,(9984,28,28,1))

**Importar tensorflow 2 (ya incluye Keras)**

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

print('Versión de Tensorflow: ' + tf.__version__)

Versión de Tensorflow: 2.8.0


In [ ]:
tf.random.set_seed(200)
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Activation('softmax'))
model.summary()

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
# checkpoint
cp_path='/content/gdrive/MyDrive/Colab Notebooks'
gpu_callback =tf.keras.callbacks.ModelCheckpoint(cp_path, 
     monitor='val_acc',save_best_only=True,save_weights_only=True,verbose=1)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_6 (Batc  (None, 28, 28, 1)        4         
 hNormalization)                                                 
                                                                 
 conv2d_6 (Conv2D)           (None, 28, 28, 64)        1664      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 dropout_8 (Dropout)         (None, 14, 14, 64)        0         
                                                                 
 batch_normalization_7 (Batc  (None, 14, 14, 64)       256       
 hNormalization)                                                 
                                                      

**Verificar disponibilidad de la GPU**


In [ ]:
nombre_gpu = tf.test.gpu_device_name()
if nombre_gpu != '/device:GPU:0':
 raise SystemError('GPU no encontrada')
print('GPU encontrada: {}'.format(nombre_gpu))  

GPU encontrada: /device:GPU:0


In [ ]:
import timeit
def entrenamiento_gpu():
  with tf.device('/device:GPU:0'):
    model.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=128,callbacks=[gpu_callback],epochs=2,verbose=1)
  
  return None
gpu_time = timeit.timeit('entrenamiento_gpu()', number=1, setup='from __main__ import entrenamiento_gpu')

Epoch 1/2
468/468 [==============================] - 20s 41ms/step - loss: 0.6832 - accuracy: 0.7834 - val_loss: 0.3780 - val_accuracy: 0.8668
Epoch 2/2
468/468 [==============================] - 19s 41ms/step - loss: 0.3906 - accuracy: 0.8614 - val_loss: 0.3371 - val_accuracy: 0.8836


In [ ]:
print('Tiempo de entrenamiento ' + str(gpu_time)+ ' segundos')

Tiempo de entrenamiento 41.805426968999654 segundos
